In [2]:
import dask.dataframe as dd
import dask.bag as da
import os
import json
import dask

from dask.distributed import LocalCluster
# dask.config.set(scheduler="threads")
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

2025-02-01 15:33:59,030 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 16381d66fc48b425ec8cc67e5c04df4c initialized by task ('shuffle-transfer-16381d66fc48b425ec8cc67e5c04df4c', 43) executed on worker tcp://127.0.0.1:60654
2025-02-01 15:34:00,686 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 16381d66fc48b425ec8cc67e5c04df4c deactivated due to stimulus 'task-finished-1738420440.6856'


In [3]:
cluster.dashboard_link

'http://127.0.0.1:8787/status'

In [ ]:
# os.makedirs('data_ex1', exist_ok=True)              # Create data/ directory

# b = dask.datasets.make_people(npartitions=100, records_per_partition=10000)                 # Make records of people
# b.map(json.dumps).to_textfiles('data_ex1/*.json')   # Encode as JSON, write to disk

In [4]:
people_bag = da.read_text('data/*.json').map(json.loads)
people_bag

dask.bag<loads, npartitions=100>

In [5]:
people_df = people_bag.to_dataframe().persist()
people_df

,age,name,occupation,telephone,address,credit-card
npartitions=100,,,,,,
,int64,object,object,object,object,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [6]:
people_df.head()

,age,name,occupation,telephone,address,credit-card
0,15,"[Jamar, Payne]",Relocation Agent,+1-914-219-7795,"{'address': '242 Kent Plaza', 'city': 'Lawndale'}","{'number': '4287 6644 0252 0398', 'expiration-..."
1,35,"[Monet, Oliver]",Administration Manager,+14101190553,"{'address': '638 Juanita Bridge', 'city': 'Dal...","{'number': '5103 5221 0018 9560', 'expiration-..."
2,89,"[Elin, Phelps]",Analyst,+19714172205,"{'address': '758 Davenport Avenue', 'city': 'V...","{'number': '4724 0787 5204 0413', 'expiration-..."
3,50,"[Brigid, Potter]",Chartered Accountant,+15738456571,"{'address': '444 Sycamore Mews', 'city': 'Kirk...","{'number': '3701 767686 99035', 'expiration-da..."
4,51,"[Jasper, Clark]",Zoologist,+19355219495,"{'address': '1373 Alemany Station', 'city': 'H...","{'number': '4637 4294 9933 1225', 'expiration-..."


In [7]:
people_bag.map(lambda x: x["age"]).mean().compute()

59.990443

In [9]:
max(people_bag.map(lambda x: x['name'][0]).frequencies().compute(), key=lambda x: x[1])

('Lane', 716)

In [12]:
people_df.drop('name', axis=1).groupby('occupation').agg({'age': 'mean'}).compute()

,age
occupation,
Proprietor,58.228736
Probation Officer,60.484524
Quality Controller,58.478992
Soldier,58.915909
Keep Fit Instructor,58.935218
...,...
Window Cleaner,62.344098
Night Watchman,61.026406
Museum Attendant,60.147219


In [13]:
people_bag.take(1)

({'age': 15,
  'name': ['Jamar', 'Payne'],
  'occupation': 'Relocation Agent',
  'telephone': '+1-914-219-7795',
  'address': {'address': '242 Kent Plaza', 'city': 'Lawndale'},
  'credit-card': {'number': '4287 6644 0252 0398',
   'expiration-date': '09/21'}},)

In [19]:
card_expering = people_bag.filter(lambda x: x['credit-card']['expiration-date'].split("/")[1] == "25")

In [20]:
card_expering

dask.bag<filter-lambda, npartitions=100>

In [21]:
card_expering.count().compute()

99905